In [ ]:
!python --version

In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
from pycaret.classification import *

# Dataset

In [ ]:
train = pd.read_csv("../input/santander-customer-satisfaction/train.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.TARGET.value_counts()

In [ ]:
train.isnull().values.sum()

In [ ]:
def process_feat(df):
    df = df.drop(["ID"], axis=1)
    return df

In [ ]:
train = process_feat(train)

# Classification

In [ ]:
setup(
    data=train,
    target='TARGET',
    imputation_type="iterative",
    fold=5,
    fix_imbalance=True,
    use_gpu=True,
    silent=True
)

In [ ]:
get_config("X_train")

In [ ]:
top5 = compare_models(sort="AUC", n_select=5)

In [ ]:
top5

In [ ]:
# xgboost = create_model('xgboost', fold=5)

In [ ]:
tuned_top5 = []
for model in top5:
    tuned = tune_model(
        model,
        fold=5,
        optimize="AUC"
    )
    tuned_top5.append(tuned)

In [ ]:
plot_model(tuned_top5[0])

In [ ]:
plot_model(tuned_top5[0], plot="error")

In [ ]:
plot_model(tuned_top5[0], plot="confusion_matrix")

In [ ]:
plot_model(tuned_top5[0], fold=5, plot="learning")

In [ ]:
plot_model(tuned_top5[0], plot="feature")

In [ ]:
plot_model(tuned_top5[0], plot="boundary")

In [ ]:
plot_model(tuned_top5[0], fold=5, plot="manifold")

In [ ]:
blended = blend_models(
    estimator_list=tuned_top5, 
    fold=5,
    optimize="AUC",
    method='soft'
)

In [ ]:
stacked = stack_models(
    estimator_list=[blended] + tuned_top5[1:], 
    meta_model=tuned_top5[0],
    fold=5,
    optimize="AUC",
    restack=False
)

In [ ]:
calibrated_stacked = calibrate_model(
    stacked,
    fold=5
)

# Soumission

In [ ]:
test = pd.read_csv("../input/santander-customer-satisfaction/test.csv")

In [ ]:
test = process_feat(test)

In [ ]:
predictions = predict_model(blended, data=test, raw_score=True)
predictions.head()

In [ ]:
sample_submission = pd.read_csv("../input/santander-customer-satisfaction/sample_submission.csv")

In [ ]:
sample_submission['TARGET'] = predictions['Score_1']
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
! kaggle competitions submit -c santander-customer-satisfaction -f submission.csv